<a href="https://colab.research.google.com/github/ambideXtrous9/Finetune-Qwen3-using-Unsloth/blob/main/Finetune_Qwen3_using_Unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install -qq sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install -qq --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.4.4 requi

In [2]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Load the base model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-0.6B",
    max_seq_length = 2048,   # Define context length
    load_in_4bit = True,     # Enable 4-bit quantization
    # token = "hf_...",      # Add your token if using a gated model
)

==((====))==  Unsloth 2025.4.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/576M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

In [4]:
# Add LoRA adapters to the model
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # LoRA rank (higher rank = more parameters, potentially better fit but more memory)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", # Target attention and MLP layers
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Scaling factor (often set to r or 2*r)
    lora_dropout = 0, # Dropout probability for LoRA layers
    bias = "none",    # Fine-tuning bias terms ('none' is often optimal)
    # Use Unsloth's gradient checkpointing for memory saving
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False, # Rank Stable LoRA (optional)
    loftq_config = None, # LoftQ initialization (optional)
)

Unsloth 2025.4.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
from datasets import load_dataset
reasoning_dataset = load_dataset("unsloth/OpenMathReasoning-mini", split = "cot")
non_reasoning_dataset = load_dataset("mlabonne/FineTome-100k", split = "train")
print("Reasoning Dataset Example Row:")
print(reasoning_dataset[0])
print("\nNon-Reasoning Dataset Example Row (raw):")
print(non_reasoning_dataset[0])

README.md:   0%|          | 0.00/603 [00:00<?, ?B/s]

cot-00000-of-00001.parquet:   0%|          | 0.00/106M [00:00<?, ?B/s]

Generating cot split:   0%|          | 0/19252 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Reasoning Dataset Example Row:
{'expected_answer': '14', 'problem_type': 'has_answer_extracted', 'problem_source': 'aops_c4_high_school_math', 'generation_model': 'DeepSeek-R1', 'pass_rate_72b_tir': '0.96875', 'problem': 'Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.', 'generated_solution': "<think>\nOkay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifyin

In [6]:
def generate_reasoning_conversation(examples):
    problems  = examples["problem"]
    # The 'generated_solution' contains the Chain-of-Thought reasoning
    solutions = examples["generated_solution"]
    conversations = []
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            # The solution here includes the <think>...</think> block already formatted
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }


In [7]:
# Apply the chat template (without tokenizing yet)
# The generated_solution in the dataset ALREADY contains the <think> tags properly formatted
reasoning_formatted_texts = tokenizer.apply_chat_template(
    reasoning_dataset.map(generate_reasoning_conversation, batched = True)["conversations"],
    tokenize = False,
)
print("\nFirst formatted Reasoning Row:")
print(reasoning_formatted_texts[0])

Map:   0%|          | 0/19252 [00:00<?, ? examples/s]


First formatted Reasoning Row:
<|im_start|>user
Given $\sqrt{x^2+165}-\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.<|im_end|>
<|im_start|>assistant
<think>
Okay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.

First, let me write down the equation again to make sure I have it right:

√(x² + 165) - √(x² - 52) = 7.

Okay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:

√(x² + 165) = 7 + √(x² - 52).

Now, if I square both sides, maybe I can get rid of the square roots. Let's do that:

(√(x² + 165))² = (7 + √(x² - 52))².

Simplifying the left side:

x² + 165 = 49 + 14√(x² - 52) + (√(x² - 52))².

The right side is expanded using the formula (a + b)² = a² + 2ab + b². So the right side becomes 7² + 2*7*√(x² - 52) +

In [8]:
from unsloth.chat_templates import standardize_sharegpt
# Standardize the ShareGPT format first (if applicable)
standardized_non_reasoning = standardize_sharegpt(non_reasoning_dataset)
# Apply the chat template
non_reasoning_formatted_texts = tokenizer.apply_chat_template(
    standardized_non_reasoning["conversations"],
    tokenize = False,
)
print("\nFirst formatted Non-Reasoning Row:")
print(non_reasoning_formatted_texts[0])

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]


First formatted Non-Reasoning Row:
<|im_start|>user
Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. 

Furthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.

Finally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.<|im_end|>
<|im_star

In [9]:
import pandas as pd
from datasets import Dataset

In [10]:
# Define desired chat data percentage
chat_percentage = 0.75 # Aim for 75% chat data
# Convert to Pandas Series for easier sampling
reasoning_series = pd.Series(reasoning_formatted_texts)
non_reasoning_series = pd.Series(non_reasoning_formatted_texts)
# Sample non-reasoning data based on the desired ratio relative to reasoning data
# Calculate how many non-reasoning samples we need
num_non_reasoning_samples = int(len(reasoning_series) * (chat_percentage / (1.0 - chat_percentage)))
# Ensure we don't request more samples than available
num_non_reasoning_samples = min(num_non_reasoning_samples, len(non_reasoning_series))

print(f"Using {len(reasoning_series)} reasoning samples.")
print(f"Sampling {num_non_reasoning_samples} non-reasoning samples.")

Using 19252 reasoning samples.
Sampling 57756 non-reasoning samples.


In [11]:
non_reasoning_subset = non_reasoning_series.sample(
    n = num_non_reasoning_samples,
    random_state = 2407, # for reproducibility
)

# Combine the datasets
combined_series = pd.concat([reasoning_series, non_reasoning_subset])
combined_series.name = "text" # The SFTTrainer expects this column name

# Convert back to Hugging Face Dataset and shuffle
combined_dataset = Dataset.from_pandas(pd.DataFrame(combined_series))
combined_dataset = combined_dataset.shuffle(seed = 3407)

print(f"\nFinal Combined Dataset size: {len(combined_dataset)}")
print("Example entry from combined dataset:")
print(combined_dataset[0]['text'])


Final Combined Dataset size: 77008
Example entry from combined dataset:
<|im_start|>user
How does the immune system respond to a viral infection in humans?<|im_end|>
<|im_start|>assistant
<think>

</think>

The immune system responds to a viral infection in humans through a complex series of events involving various cells, proteins, and signaling molecules. The response can be divided into two main categories: the innate immune response and the adaptive immune response.

1. Innate immune response: This is the first line of defense against viral infections and acts rapidly to limit the spread of the virus. The key components of the innate immune response include:

a. Physical barriers: The skin and mucous membranes act as physical barriers, preventing the entry of viruses into the body.

b. Antiviral proteins: Upon viral infection, cells produce and release proteins called interferons, which have antiviral properties. Interferons signal neighboring cells to increase their antiviral def

In [12]:
from trl import SFTTrainer, SFTConfig

In [13]:
sftconfig = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Effective batch size = 2 * 4 = 8
        warmup_steps = 5,
        max_steps = 30,                 # Short run for demonstration; set to None for full epochs
        # num_train_epochs = 1,         # Alternatively, train for 1 full epoch
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(), # Use bf16 if available, else fp16
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",           # Use 8-bit AdamW optimizer
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",         # Directory to save checkpoints
        report_to = "none",             # Disable external reporting (like WandB) for this example
    )

In [14]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    dataset_text_field = "text", # Column name we created
    max_seq_length = 2048,      # Should match model loading
    args = sftconfig
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/77008 [00:00<?, ? examples/s]

In [15]:
# Start training
print("Starting training...")
trainer_stats = trainer.train()
print("Training finished.")
# You can print training stats if needed
# print(trainer_stats)

Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 77,008 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 20,185,088/6,000,000,000 (0.34% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.030100
2,0.939000
3,0.902600
4,1.358200
5,0.969300
6,0.985600
7,1.030200
8,1.249300
9,0.995700
10,0.887000


Training finished.


In [16]:
from transformers import TextStreamer
messages = [
    {"role" : "user", "content" : "Solve (x + 2)^2 = 0."}
]

In [17]:
# Format the prompt, explicitly DISABLING thinking mode
text_input_no_think = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Crucial for generation
    enable_thinking = False,      # *** Disable thinking ***
)


print("--- Non-Thinking Inference ---")
print("Formatted Input:\n", text_input_no_think)

--- Non-Thinking Inference ---
Formatted Input:
 <|im_start|>user
Solve (x + 2)^2 = 0.<|im_end|>
<|im_start|>assistant
<think>

</think>




In [18]:
# Generate response using parameters suitable for non-thinking/chat
inputs = tokenizer(text_input_no_think, return_tensors = "pt").to("cuda")
streamer_no_think = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    **inputs,
    max_new_tokens = 256,
    temperature = 0.7, # Recommended for chat
    top_p = 0.8,       # Recommended for chat
    top_k = 20,
    streamer = streamer_no_think,
    eos_token_id = tokenizer.eos_token_id # Ensure generation stops properly
)
print("\n-----------------------------")

To solve the equation (x + 2)^2 = 0, we can follow these steps:

1. Take the square root of both sides of the equation to get rid of the square on the left side. This will give us:
   x + 2 = ±√0

2. Since the square root of 0 is 0, the equation becomes:
   x + 2 = 0 or x + 2 = -0

3. Simplify the equation by subtracting 2 from both sides:
   x = -2 or x = -2

4. Therefore, the solutions to the equation (x + 2)^2 = 0 are x = -2.<|im_end|>

-----------------------------


## **Thinking Inference:**

In [19]:
# Format the prompt, explicitly ENABLING thinking mode
text_input_think = tokenizer.apply_chat_template(
    messages, # Same user message
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = True,       # *** Enable thinking ***
)

print("--- Thinking Inference ---")
print("Formatted Input:\n", text_input_think)

--- Thinking Inference ---
Formatted Input:
 <|im_start|>user
Solve (x + 2)^2 = 0.<|im_end|>
<|im_start|>assistant



In [20]:
# Generate response using parameters suitable for thinking/reasoning
inputs_think = tokenizer(text_input_think, return_tensors = "pt").to("cuda")
streamer_think = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    **inputs_think,
    max_new_tokens = 1024, # Allow more tokens for reasoning steps
    temperature = 0.6,   # Recommended for reasoning
    top_p = 0.95,        # Recommended for reasoning
    top_k = 20,
    streamer = streamer_think,
    eos_token_id = tokenizer.eos_token_id # Ensure generation stops properly
)
print("\n-----------------------------")

<think>

</think>

To solve the equation $(x + 2)^2 = 0$, we start by recognizing that the square of a number is zero only when the number itself is zero. This is a fundamental property of squares. Therefore, we can rewrite the equation as $(x + 2)^2 = 0$ and substitute $x = -2$ into the equation to find the solution.

Substituting $x = -2$ into the equation gives:
$$
(-2 + 2)^2 = 0
$$
Simplifying the left-hand side:
$$
0^2 = 0
$$
Since the left-hand side equals the right-hand side, the solution is correct. Therefore, the solution to the equation $(x + 2)^2 = 0$ is $x = -2$.

To verify the solution, we can plug $x = -2$ back into the original equation:
$$
(-2 + 2)^2 = 0
$$
Simplifying the left-hand side:
$$
0^2 = 0
$$
Since the left-hand side equals the right-hand side, the solution is verified. Therefore, the solution to the equation $(x + 2)^2 = 0$ is $x = -2$.<|im_end|>

-----------------------------


In [21]:
# Save LoRA adapters locally
model.save_pretrained("qwen3_0.6b_reasoning_chat_lora")
tokenizer.save_pretrained("qwen3_0.6b_reasoning_chat_lora")

print("LoRA adapters saved locally to 'qwen3_0.6b_reasoning_chat_lora'")

# Optional: Push to Hugging Face Hub
# model.push_to_hub("your_username/qwen3_14b_reasoning_chat_lora", token="YOUR_HF_TOKEN")
# tokenizer.push_to_hub("your_username/qwen3_14b_reasoning_chat_lora", token="YOUR_HF_TOKEN")

# To load these adapters later:
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "qwen3_14b_reasoning_chat_lora", # Path to saved adapters
#     load_in_4bit = True,
# )

LoRA adapters saved locally to 'qwen3_0.6b_reasoning_chat_lora'
